In [4]:
from dataclasses import dataclass

In [3]:
from serde import deserialize, serialize
from serde.json import from_json

In [13]:
import pandas as pd

In [26]:
import re

In [1]:
with open("brute.json", "r") as fichier:
    data = fichier.read()

In [2]:
type(data)

str

In [5]:
@serialize
@deserialize
@dataclass 
class Annonce:
    id: str
    genre: str
    prix: str
    pcs: str
    desc: str
    lien: str

In [6]:
annonces = from_json(list[Annonce], data)

JSONDecodeError: Extra data: line 2 column 1 (char 445)

In [7]:
data = data.replace("}", "},")

In [8]:
data = "[" + data + "]"

In [9]:
annonces = from_json(list[Annonce], data)

JSONDecodeError: Expecting value: line 1819 column 1 (char 815182)

In [10]:
with open("data.json", "w") as fichier:
    fichier.write(data)

In [11]:
with open("data.json", "r") as fichier:
    data = fichier.read()

In [88]:
annonces = from_json(list[Annonce], data)

In [154]:
df = pd.DataFrame(annonces)

In [155]:
df.head()

,id,genre,prix,pcs,desc,lien
0,annonce-138905473-376235,Appartement,374 400 €,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...
1,annonce-140620177-376235,Appartement,499 200 €,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...
2,annonce-140620179-376235,Appartement,499 200 €,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...
3,annonce-133494153-376235,Maison / Villa,508 000 €,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...
4,annonce-137425993-376235,Maison / Villa,676 000 €,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...


In [156]:
df.dtypes

id       object
genre    object
prix     object
pcs      object
desc     object
lien     object
dtype: object

# Exercice

Faites le nettoyage de toutes les colonnes sauf desc.

## Colonne id

In [157]:
df.id.count()

1818

In [158]:
df.id.unique().shape

(1798,)

In [159]:
df.drop_duplicates("id", inplace=True)

In [160]:
df.id.count()

1798

## Colonne genre 

In [161]:
df.genre.unique()

array(['Appartement', 'Maison / Villa', 'Appartement neuf',
       'Projet de construction', 'Maison / Villa neuve', 'Bâtiment',
       'Parking', 'Bureau', 'Local commercial', 'Immeuble', 'Boutique',
       'Terrain', 'Divers', 'Loft/Atelier/Surface', 'Château',
       'Hôtel particulier'], dtype=object)

In [162]:
df.genre.value_counts()

Appartement               1037
Maison / Villa             376
Appartement neuf           204
Parking                     38
Immeuble                    32
Local commercial            30
Château                     17
Maison / Villa neuve        16
Bureau                      14
Boutique                    14
Bâtiment                     5
Divers                       5
Projet de construction       4
Terrain                      3
Loft/Atelier/Surface         2
Hôtel particulier            1
Name: genre, dtype: int64

In [163]:
# Suppression des majuscules
df.genre = df.genre.str.lower()

In [164]:
df.genre.unique()

array(['appartement', 'maison / villa', 'appartement neuf',
       'projet de construction', 'maison / villa neuve', 'bâtiment',
       'parking', 'bureau', 'local commercial', 'immeuble', 'boutique',
       'terrain', 'divers', 'loft/atelier/surface', 'château',
       'hôtel particulier'], dtype=object)

In [165]:
motif = re.compile("maison|château|particulier")
df["maison"] = df.genre.str.contains(motif)

In [166]:
df["appartement"] = df.genre.str.contains("appartement")

In [167]:
motif = re.compile("neuf|neuve")
df["neuf"] = df.genre.str.contains(motif)

In [168]:
masque = df.appartement | df.maison
masque

0        True
1        True
2        True
3        True
4        True
        ...  
1813     True
1814     True
1815     True
1816    False
1817    False
Length: 1798, dtype: bool

In [169]:
df = df[masque]
df.describe()

,id,genre,prix,pcs,desc,lien,maison,appartement,neuf
count,1651,1651,1651,1651,1651,1651,1651,1651,1651
unique,1651,6,1074,1045,1447,1651,2,2,2
top,annonce-138905473-376235,appartement,250 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.seloger.com/annonces/achat/apparte...,False,True,False
freq,1,1037,10,30,19,1,1241,1241,1431


In [170]:
df = df.drop("genre", axis=1)

In [171]:
df.describe()

,id,prix,pcs,desc,lien,maison,appartement,neuf
count,1651,1651,1651,1651,1651,1651,1651,1651
unique,1651,1074,1045,1447,1651,2,2,2
top,annonce-138905473-376235,250 000 €,3 p 2 ch 65 m²,"Livraison: Octobre 2020 - Rare, plein centre d...",https://www.seloger.com/annonces/achat/apparte...,False,True,False
freq,1,10,30,19,1,1241,1241,1431


## Colonne Prix

In [172]:
df.prix = df.prix.str.replace("HH", "").str.replace(" ", "").str.replace("€", "")

In [173]:
df = df[(df.prix != "")]

In [174]:
df["prix"] = df.prix.astype(int)

In [175]:
df.dtypes

id             object
prix            int64
pcs            object
desc           object
lien           object
maison           bool
appartement      bool
neuf             bool
dtype: object

## Colonne pcs

In [176]:
df.head()

,id,prix,pcs,desc,lien,maison,appartement,neuf
0,annonce-138905473-376235,374400,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
1,annonce-140620177-376235,499200,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
2,annonce-140620179-376235,499200,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
3,annonce-133494153-376235,508000,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...,True,False,False
4,annonce-137425993-376235,676000,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...,True,False,False


In [177]:
masque = (df.pcs.str.replace(" ","").str.match("(.*)p(.*)ch(.*)m².*") == False)
df.pcs.str.replace(" ","")[masque]

18       2p31,6m²1balc
19      3p60,73m²1balc
28              1p29m²
34         1p25m²1tess
41       3p61,96m²tess
             ...      
1804     1p13,77m²3etg
1805        1p26m²1asc
1807          3p66,3m²
1814          4p84,4m²
1815          4p84,4m²
Name: pcs, Length: 183, dtype: object

In [178]:
masque = (df.pcs.str.replace(" ","").str.match("^([0-9]+)p.*?([0-9,]+)m².*?$"))
df = df[masque]

In [179]:
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)
df.head()

,id,prix,pcs,desc,lien,maison,appartement,neuf
0,annonce-138905473-376235,374400,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
1,annonce-140620177-376235,499200,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
2,annonce-140620179-376235,499200,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...,False,True,False
3,annonce-133494153-376235,508000,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...,True,False,False
4,annonce-137425993-376235,676000,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...,True,False,False


In [180]:
res = df.pcs.str.replace(" ", "").str.extractall("([0-9]+)p.*?([0-9,]+)m²")


In [181]:
res.head()

,,0,1
,match,,
0,0,3,90
1,0,5,"146,27"
2,0,5,110
3,0,6,132
4,0,7,185


In [182]:
res.reset_index(inplace=True)
res

,level_0,match,0,1
0,0,0,3,90
1,1,0,5,"146,27"
2,2,0,5,110
3,3,0,6,132
4,4,0,7,185
...,...,...,...,...
1638,1638,0,3,"66,3"
1639,1639,0,2,42
1640,1640,0,3,76
1641,1641,0,4,"84,4"


In [186]:
df["nombre_pieces"] = res[0].astype(int)

In [184]:
df["surface"] = res[1].str.replace(",", ".").astype(float)

In [187]:
df.head()

,id,prix,pcs,desc,lien,maison,appartement,neuf,nombre_pieces,surface
0,annonce-138905473-376235,374400,3 p 2 ch 90 m²,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,https://www.seloger.com/annonces/achat/apparte...,False,True,False,3,90.00
1,annonce-140620177-376235,499200,"5 p 4 ch 146,27 m²",TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,https://www.seloger.com/annonces/achat/apparte...,False,True,False,5,146.27
2,annonce-140620179-376235,499200,5 p 3 ch 110 m²,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,https://www.seloger.com/annonces/achat/apparte...,False,True,False,5,110.00
3,annonce-133494153-376235,508000,6 p 4 ch 132 m²,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,https://www.seloger.com/annonces/achat/maison/...,True,False,False,6,132.00
4,annonce-137425993-376235,676000,7 p 5 ch 185 m²,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,https://www.seloger.com/annonces/achat-de-pres...,True,False,False,7,185.00


In [188]:
df.dtypes

id                object
prix               int64
pcs               object
desc              object
lien              object
maison              bool
appartement         bool
neuf                bool
nombre_pieces      int64
surface          float64
dtype: object

In [189]:
df.drop(["pcs", "id", "lien"], axis=1, inplace=True)

## Colonne desc

In [190]:
df.head()

,prix,desc,maison,appartement,neuf,nombre_pieces,surface
0,374400,Appartement type 3 - TOURS CATHÉDRALE TOURS CA...,False,True,False,3,90.00
1,499200,TOURS HYPERCENTRE - Appartement TOURS HYPERCEN...,False,True,False,5,146.27
2,499200,TOURS PRÉBENDES NORD - APPARTEMENT TOURS PRÉBE...,False,True,False,5,110.00
3,508000,TOURS PRÉBENDES - PARTICULIER TOURANGEAUX TOUR...,True,False,False,6,132.00
4,676000,TOURS STRASBOURG / RABELAIS - Maison TOURS STR...,True,False,False,7,185.00


In [191]:
desc = df.desc.copy()

In [192]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.feature_extraction.text import 